Sprintの目的
スクラッチを通してSVMを理解する
線形モデルと異なる手法に触れる


2.SVMスクラッチ

分類のためのサポートベクターマシン（SVM、サポートベクトルマシン）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


SVMには学習時に分類の間違いを認めるソフトマージンSVMと、認めないハードマージンSVMがありますが、ここでは実装が単純なハードマージンSVMを扱います。


以下に雛形を用意してあります。このScratchSVMClassifierクラスにコードを書き加えていってください。



【問題1】ラグランジュの未定乗数法による最急降下
SVMの学習は、ラグランジュの未定乗数法を用います。サンプル数分のラグランジュ乗数 
λ
 を用意して、以下の式により更新していきます。この計算を行うメソッドをScratchSVMClassifierクラスに実装してください。



$$
λ_{i}^{new}=λ_i+α(1−\sum_{i=1}^{m}λ_jy_iy_jk(x_i,x_j))
$$

ここで 
$$
k(x_i,x_j)
$$
 はカーネル関数です。線形カーネルの場合は次のようになります。他のカーネル関数にも対応できるように、この部分は独立したメソッドとしておきましょう。

$$
k(x_i,x_j)=x_i^Tx_j
$$

条件として、更新毎に 
λi>=0
を満たす必要があります。満たさない場合は 
λi=0
とします。


i,j : サンプルのインデックス


$$
λ_{i}^{new}: 更新後のi番目のサンプルのラグランジュ乗数
$$

λi: 更新前のi番目のサンプルのラグランジュ乗数


α
 : 学習率


λ
j
 : j番目のサンプルのラグランジュ乗数


y
i
 : i番目のサンプルのラベル


y
j
 : j番目のサンプルのラベル


x
i
 : i番目のサンプルの特徴量ベクトル


x
j
 : j番目のサンプルの特徴量ベクトル


あるサンプルに対してのすべてのサンプルとの関係を計算していくことになります。




In [121]:
class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装
    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue
    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル
    """
    # def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
    def __init__(self, num_iter, lr, kernel, threshold, verbose):
        # ハイパーパラメータを属性として記録
        # print("num_iter")
        # print(num_iter)
        # print("kernel")
        # print(kernel)
        # print("threshold")
        # print(threshold)
        # print("verbose")
        # print(verbose)
        self.iter = num_iter 
        self.lr = lr 
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
        self.lamda = np.array([])  # 初期値
        self.lam_sv = np.array([])  # サポートベクターの保存する変数初期値
        self.X_sv = np.array([]) #サポートベクターのn番目のラベル初期値
        self.y_sv = np.array([]) #サポートベクターのn番目の特徴量初期値
        self.sv_pred = np.array([]) #推定値を保存する変数初期値

    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        # print("X1")
        # print(X)
        # print(X.shape)
        # print("y")
        # print(y)
        # print(y.shape)

        # m = X.shape[0]
        # hoge = np.ones((m, 1))
        # X = np.hstack((hoge,X)) #１がいらない

        # m2 = X_val.shape[0]
        # hoge2 = np.ones((m2, 1))
        # X_val = np.hstack((hoge2,X_val))
                            
        self.lamda = np.zeros(X.shape[0]) #X.shape[1]：行列Xの列数を表す。np.zeros(X.shape[1])で、列数がX.shape[1]である０だけで構成される行列を作成している
        self.sv_pred = np.zeros(X.shape[0])
      

        # メイン処理
        for k in range(self.iter):
          kern = self._kernel2(X,X) #カーネル関数
          # 問題1解答（ラグランジュの未定乗数法による最急降下）
          # self.lamda = self._svm_gradient_descent(X,y,kern)
          self.lamda = self._svm_gradient_descent(X,y)
          # print("self.lamda1")
          # print(self.lamda)
          # print(self.lamda.shape)
          # print(len(self.lamda))
          # print("self.verbose2")
          # print(self.verbose)
          if(self.verbose ==True):# 問題2解答（サポートベクターの決定）
            # print("self.verbose")
            # print(self.verbose)
            mmm = np.where(self.lamda >= self.threshold)  #if(self.lamda>=10**-5):を元々書いたがこれだと処理できない
            #mmm = np.where(self.lamda >= self.threshold) 閾値超えるself.lamdaのインデックスをmmmに代入している
            # print("mmm")
            # print(mmm)
            # print("np.where(self.lamda >= self.threshold)")
            # print(np.where(self.lamda >= self.threshold))
            self.lam_sv = np.append( self.lam_sv, self.lamda[mmm] )
            # print("self.lam_sv")
            # print(self.lam_sv)
            # print(self.lam_sv.shape)
            # print(len(self.lam_sv))
            self.X_sv = np.append( self.X_sv, X[mmm] )
            # print("self.X_sv")
            # self.X_sv = self.X_sv[::100]
            # print(self.X_sv)
            # print(len(self.X_sv))
            # print(self.X_sv.shape)
            self.y_sv = np.append( self.y_sv, y[mmm] )
            # print("self.y_sv")
            # print(self.y_sv)
            # print(len(self.y_sv))
            # print(self.y_sv.shape)

 

    #カーネル関数実装
    def _kernel2(self,X1,X2):
      """
      カーネル関数を計算する
      Parameters
      ----------
      X : 次の形のndarray, shape (n_samples, n_features)
        訓練データ
      Returns
      -------
        次の形のndarray, shape (n_samples, 1)
        カーネル関数による推定結果
      """
      # print(X)
      # print("X1")
      # print(X1.shape)
      # print("X2")
      # print(X2.shape)
      kern = X1.T @ X2
      # print("kern")
      # print(kern)
      # print(kern.shape)
      return kern
      #return = kernel3
      # if(self.kernel=='linear'):
      #   kernel3 = X @ X
      #   return = kernel3
      # else:
      #   pass #取り敢えず、カーネル関数以外でも設定できるようにしたが、設定が""linear(線形)"だから使用することはない
    # return = kernel2
    
    # 問題１解答
    # def _svm_gradient_descent(self,X,y,kern):
    def _svm_gradient_descent(self,X,y):
        """
        最急降下法によるパラメータの更新値計算
        適当な重みIrを掛けてλが小さくなる方へ徐々に転がっていくようにする
         X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
         y_true : 入力するモデルの学習に使用する正解値
        Returns
        -------
        λである。
        """
        m=X.shape[0] #n_samplesサンプル数
        # n=X.shape[1] #n_features特徴量の数
        # print("X111")
        # print(X.shape)
        # print("X222")
        # print(X.T)
        # print((X.T).shape)
        # print("y")
        # print(y)
        # print(y.shape)
        for i in range(m):
          gradient = 0
          for j in range(m):
            # print("self.lamda[j]")
            # print(self.lamda)
            # print(self.lamda.shape)
            # print("y")
            # print(y)
            # print(y.shape)
            # print("X.T")
            # print(X.T)
            # print(X.T.shape)
            # print(type(X.T))
            # print("X")
            # print(X)
            # print(X.shape)
            # print(type(X))
            gradient += (self.lamda[j] * y[i] * y[j] * (X[i].T @ X[j]))
            # print("gradient")
            # print(gradient)
            # print(gradient.shape)
          self.lamda[i] +=  self.lr * (1-gradient)
          # print("self.lamda[i]")
          # print(self.lamda[i])
          # print(self.lamda.shape)

          # self.lamda[i] = round(self.lamda[i])
          if(self.lamda[i]>=0):
          # self.lamda[i] +=  self.lr * (1-gradient)
            self.lamda = self.lamda
            # print("self.lamda3")
            # print(self.lamda[i])
          else:
            self.lamda = 0
          # print("gradient")
          # print(gradient)
          # print("self.lamda")
          # print(self.lamda)
          return self.lamda


    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル
        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """
        N = len(self.lam_sv)
        print("N = len(self.lam_sv)")
        print(N)
        sv_pre = 0
        # self.X_sv = self.X_sv[0:100]
        m = self.X_sv.shape[0]
        print("m")
        print(m)
        hoge = np.ones((m, 1))
        # print("hoge")
        # print(hoge)
        print("self.X_sv")
        print(self.X_sv)
        print(len(self.X_sv))
        print(self.X_sv.shape)
        # print(self.X_sv.shape)
        # self.X_sv = self.X_sv.reshape(m,1)
        # self.X_sv = np.hstack((hoge,self.X_sv))
        print("X")
        print(X)
        print(X.shape)

        for n in range(N):
          # print("self.lam_sv")
          # print(self.lam_sv)
          # print(len(self.lam_sv))
          # print(self.lam_sv.shape)
          # print("self.y_sv")
          # print(self.y_sv)
          # print(len(self.y_sv))
          # print(self.y_sv.shape)
          # print("X")
          # print(X)
          # print(len(X))
          # print(X.shape)
          
          
          # print("self.X_sv2")
          # print(self.X_sv)
          # print(len(self.X_sv))
          # print(self.X_sv.shape)
          # print("sv_pre")
          # # print(len(sv_pre))
          # print(sv_pre)
          # sv_pre += self.lam_sv[n]*self.y_sv[n]* self._kernel2(X,self.X_sv)
          # sv_pre += self.lam_sv[n]*self.y_sv[n]* (X @ self.X_sv.T)
          # print("self.lam_sv[n]")
          # print(self.lam_sv[n])
          # print(self.lam_sv.shape)
          # print("self.y_sv[n]")
          # print(self.y_sv[n])
          # print(self.y_sv[n].shape)
          
          # print("X[n]")
          # print(X[n])
          # print(X[n].shape)
          # print("self.X_sv[n].T")
          # print(self.X_sv[n].T)
          # print(self.X_sv[n].T.shape)
          # sv_pre += self.lam_sv[n]*self.y_sv[n]* (X[n] @ self.X_sv[n])
          sv_pre += self.lam_sv[n]*self.y_sv[n]* (X.T[n] * self.X_sv[n])
        return sv_pre

【問題2】サポートベクターの決定
計算したラグランジュ乗数 
λ
 が設定した閾値より大きいサンプルをサポートベクターとして扱います。推定時にサポートベクターが必要になります。サポートベクターを決定し、インスタンス変数として保持しておくコードを書いてください。


閾値はハイパーパラメータですが、1e-5程度からはじめると良いでしょう。サポートベクターの数を出力させられるようにしておくと学習がうまく行えているかを確認できます。




In [122]:
#問題１に解答しました

【問題3】推定

推定時には、推定したいデータの特徴量とサポートベクターの特徴量をカーネル関数によって計算します。求めた 

$$
k(x_i,x_j)=x_i^Tx_j
$$

f(x)
 の符号が分類結果です。

$$
f(x)=\sum_{n=1}^{N}λ_ny_{sv_n}k(x,s_n)
$$

x
 : 推定したいデータの特徴量ベクトル


N
 : サポートベクターの数


n
 : サポートベクターのインデックス


λ
n
 : 
n
番目のサポートベクターのラグランジュ乗数


$y_{sv_n}$ 
番目のサポートベクターのラベル


k
(
)
 : カーネル関数


s
n
 : 
n
番目のサポートベクターの特徴量




In [123]:
#問題１に解答しました

3.検証

【問題4】学習と推定
機械学習スクラッチ入門のSprintで用意したシンプルデータセット1の2値分類に対してスクラッチ実装の学習と推定を行なってください。


scikit-learnによる実装と比べ、正しく動いているかを確認してください。


AccuracyやPrecision、Recallなどの指標値はscikit-learnを使用してください。




In [124]:
#SVMのモデルで、シンプルデータセット１を使用して、学習と正答率を算出する
#シンプルデータセット1作成コード
#scikit-learnによる実装
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X1_1 = np.concatenate([f0, f1])
y1_1= np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])


#トレーニングデータとテストデータに分けて実行してみる------------------
X_train, X_test, y_train, y_test=train_test_split(X1_1, y1_1, train_size=0.8)
from sklearn.svm import SVC
# 線形SVMのインスタンスを生成
model = SVC(kernel='linear', random_state=None)

# モデルの学習。fit関数で行う。
model.fit(X_train, y_train)

from sklearn import linear_model, metrics, preprocessing #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用
#正答率を求める
pre1=model.predict(X_test)
# print("予測値=",pre1)
ac_score1=metrics.accuracy_score(y_test,pre1)
# print("正答率 = ",ac_score1)


from sklearn.metrics import classification_report
import pandas as pd
import pprint


pre1=model.predict(X_test)
y_true_multi=y_test

y_pred_multi=pre1
print(classification_report(y_true_multi, y_pred_multi)) #引数に入れるのは、下記でサイトで調べた

              precision    recall  f1-score   support

          -1       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        57

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [125]:
#スクラッチSVM実装でシンプルデータセット１を使用して、学習と正答率を算出する

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pprint
from sklearn import linear_model, metrics, preprocessing #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用

#シンプルデータセット1作成コード
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X1_1 = np.concatenate([f0, f1])
y1_1= np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

# print("X1_1")
# print(X1_1.shape)
# print("y1_1")
# print(y1_1.shape)


#トレーニングデータとテストデータに分けて実行してみる------------------
X_train, X_test, y_train, y_test=train_test_split(X1_1, y1_1, train_size=0.8)



scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


model = ScratchSVMClassifier(num_iter=100, lr=0.1, kernel='linear', threshold=1e-5, verbose=True)
# モデルの学習。fit関数で行う。
model.fit(X_train, y_train)

# sv_pre = self.predict(X)
# # print("sv_pre")
# # print(sv_pre)
# # print("self.lam_sv")
# # print(self.lam_sv)
# # print(self.lam_sv.shape)
# # print("self.y_sv")
# # print(self.y_sv)
# # print(self.y_sv.shape)
# # print("X")
# # print(X)
# # print(X.shape)
# # print("self.X_sv")
# # print(self.X_sv)
# # print(self.X_sv.shape)
# # print("sv_pre")
# # print(sv_pre)
# self.sv_pred = np.append( self.sv_pred, sv_pre ) #物凄く時間がかかった所。（空ののself.lossにnumpy ndarrayの要素を追加する方法）

#正答率を求める
print("X_test")
print(X_test.shape)

pre1=model.predict(X_test)
print("予測値=",pre1)
# ac_score1=metrics.accuracy_score(y_test,pre1)
# print("正答率 = ",ac_score1)

y_true_multi=y_test
y_pred_multi=pre1
print("y_true_multi")
print(y_true_multi)
print(y_true_multi.shape)
print("y_pred_multi")
print(y_pred_multi)
print(y_pred_multi.shape)
print(classification_report(y_true_multi, y_pred_multi)) 

X_test
(100, 2)
N = len(self.lam_sv)
100
m
200
self.X_sv
[-0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.96235097
 -0.11179853 -0.96235097 -0.11179853 -0.96235097 -0.11179853 -0.962

IndexError: ignored

まだスクラッチ実装が正しく動作してない

【問題5】決定領域の可視化
決定領域を可視化してください。


以下の例のようにサポートベクターは異なる色で示してください。

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import numpy as np
import itertools
# def decision_region(X, y, model, step=0.1, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['1','-1']):
#     """
#     2値分類を2次元の特徴量で学習したモデルの決定領域を描く。
#     背景の色が学習したモデルによる推定値から描画される。
#     散布図の点は訓練データまたは検証データである。
#     Parameters
#     ----------------
#     X : ndarray, shape(n_samples, 2)
#         特徴量
#     y : ndarray, shape(n_samples,)
#         ラベル
#     model : object
#         学習したモデルのインスンタスを入れる
#     step : float, (default : 0.1)
#         推定値を計算する間隔を設定する
#     title : str
#         グラフのタイトルの文章を与える
#     xlabel, ylabel : str
#         軸ラベルの文章を与える
#     target_names= : list of str
#         凡例の一覧を与える
#     """
#     # setting
#     scatter_color = ['red', 'blue']
#     contourf_color = ['pink', 'skyblue']
#     n_class = 2
#     # pred
#     mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
#     mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
#     # y_pred = model.predict(mesh).reshape(mesh_f0.shape)
#     y_pred = model.predict(mesh)
#     # plot
#     plt.title(title)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)
#     plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
#     plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
#     for i, target in enumerate(set(y)):
#         plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
#     patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
#     plt.legend(handles=patches)
#     plt.legend()
#     plt.show()



import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
iris_dataset = load_iris()

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import itertools

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pprint
from sklearn import linear_model, metrics, preprocessing #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用

#シンプルデータセット1作成コード
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X1_1 = np.concatenate([f0, f1])
y1_1= np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

print("X1_1")
print(X1_1.shape)
print("y1_1")
print(y1_1.shape)


#トレーニングデータとテストデータに分けて実行してみる------------------
X_train, X_test, y_train, y_test=train_test_split(X1_1, y1_1, train_size=0.8)

model = ScratchSVMClassifier(num_iter=5, lr=0.1, kernel='linear', threshold=1e-5, verbose=True)

# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train=scaler.transform(X_train)
# X_test=scaler.transform(X_test)


#標準化は「平均が0、標準偏差が1になるようにデータを加工（スケーリング）」し、正規化は「最低が0、最高が1になるようにデータを加工（スケーリング）する」モジュールと説明されています。
#標準化（standardization） と 正規化（normalization）
from sklearn.preprocessing import StandardScaler

X_train_data = X_train 
scaler = StandardScaler()
scaler.fit(X_train_data)
X_train_transformed = scaler.transform(X_train_data)

X_test_data = X_test
X_test_transformed = scaler.transform(X_test_data)

y_train_data = y_train 
# scaler.fit(y_train_data)
# y_train_transformed = scaler.transform(y_train_data)

y_test_data = y_test
# y_test_transformed = scaler.transform(y_test_data)

def decision_region(X, y, model, step=0.1, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['1','-1']):
    """
    2値分類を2次元の特徴量で学習したモデルの決定領域を描く。
    背景の色が学習したモデルによる推定値から描画される。
    散布図の点は訓練データまたは検証データである。
    Parameters
    ----------------
    X : ndarray, shape(n_samples, 2)
        特徴量
    y : ndarray, shape(n_samples,)
        ラベル
    model : object
        学習したモデルのインスンタスを入れる
    step : float, (default : 0.1)
        推定値を計算する間隔を設定する
    title : str
        グラフのタイトルの文章を与える
    xlabel, ylabel : str
        軸ラベルの文章を与える
    target_names= : list of str
        凡例の一覧を与える
    """
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    print(type(predict))
    y_pred = model.predict(mesh).reshape(mesh_f0.shape)
    # y_pred = model.predict(mesh)
    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()


#ロジスティック回帰 
#訓練データをプロット
from sklearn.linear_model import LogisticRegression
# X2_1=scaler.transform(X_train_transformed)
# y2_1=list(itertools.chain.from_iterable(y_train_data))

fig = plt.figure()
model2 = model.fit(X_train_transformed,y_train_data)
decision_region(X_train_transformed, y_train_data, model2, step=0.1, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['1','-1'])
# decision_region(X2, y2, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])

ax2 = fig.add_subplot(1, 3, 2)
fig.tight_layout()              #レイアウトの設定
plt.show()

# #検証データをプロット
# # X_test_data_list = list(itertools.chain.from_iterable(X_test_data))
# # print(X_test_data_list)
# X2_2=scaler.transform(X_test_data)
# # print(X2_2)
# y2_2=list(itertools.chain.from_iterable(y_test_data))
# # print(y2_2)
# decision_region(X2_2, y2_2, reg, step=0.01, title='Logistic regression validation data decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])
# ax3 = fig.add_subplot(1, 3, 3)
# fig.tight_layout()              #レイアウトの設定
# plt.show()


# from sklearn.metrics import classification_report
# import pandas as pd
# import pprint


# predict2 = reg.predict(X_test6)
# y_true_multi2=test_label6
# y_pred_multi2=predict2
# print(classification_report(y_true_multi2, y_pred_multi2)) #引数に入れるのは、下記でサイトで調べた




In [ ]:
# import numpy as np
# X = [10 ,10]
# print("X1")
# print(X)
# # print(X.shape)
# X = np.array(X)
# print("X2")
# print(X)
# print(X.shape)
# h = X @ X
# print(h)

In [ ]:
# #カーネル関数実装
# def _kernel2(X1,X2):
#   """
#   カーネル関数を計算する
#   Parameters
#   ----------
#   X : 次の形のndarray, shape (n_samples, n_features)
#     訓練データ
#   Returns
#   -------
#     次の形のndarray, shape (n_samples, 1)
#     カーネル関数による推定結果
#   """
#   # print(X)
#   # print("X1")
#   # print(X1.shape)
#   # print("X2")
#   # print(X2.shape)
#   kern = X1.T @ X2
#   return kern

# X1 = X_train
# print("X1")
# print(X1)
# print(X1.shape)

# X2 = X_train
# print("X2")
# print(X2)
# print(X2.shape)
# kern = _kernel2(X1,X2)
# print(kern)

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# import matplotlib.patches as mpatches
# import numpy as np
# import itertools

# import numpy as np
# import pandas as pd
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')

# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# from sklearn.svm import LinearSVC

# iris_dataset = load_iris()



# #説明変数
# iris_dataset_data = pd.DataFrame(iris_dataset.data, columns=["sepal_length","sepal_width","petal_length","petal_width"])
# iris_dataset_all = pd.concat([iris_dataset_data], axis=1)
# X = iris_dataset_all
# # print(X)
# print(type(X))
# #目的変数（花の種類）も同様にyに格納してください。
# iris_dataset_target = pd.DataFrame(iris_dataset.target, columns=["Species"])
# iris_dataset_all2 = pd.concat([iris_dataset_target], axis=1)
# y = iris_dataset_all2
# print(type(y))
# # print(y)

# # df = pd.concat([X, y], axis=1)
# # print(df)

# #sepal_lengthとpetal_length
# print("sepal_lengthとpetal_lengthのデータフレーム")
# x=X.loc[:,['sepal_length','petal_length']]
# print(type(x))
# print(x)
# # X.loc[50:149,['sepal_length','petal_length']]
# # print(X.loc[50:149,['sepal_length','petal_length']])
# # x = pd.DataFrame(iris_dataset.data, columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
# # print(x)
# # print(df_concat['sepal_width'])

# # x = x.loc[50:149, ["sepal_length", "petal_length"]]

# # y = pd.Series(iris_dataset.target, name="y")
# oldy=y[(y['Species'] == 1) | (y['Species'] == 2)]
# print(type(oldy))
# print(oldy)
# # y = pd.DataFrame(iris_dataset.target, columns=["y"])

# # print("virgicolorとvirginicaのデータフレーム")
# # y = y[(y['y'] != 0)]
# # print(y)
# # print(y.shape)
# # print(type(y))



# #PandasのDataFrameをNumPyのndarrayに変換
# print("sepal_lengthとpetal_lengthのndarray")
# x=x.loc[50:149,['sepal_length','petal_length']]
# print(type(x))
# print(x)
# print("xの型は")
# print(type(x))
# X = x.values
# print(X)
# print("Xの型は")
# print(type(X))
# # X_list=X.tolist()
# # print(X_list)
# # print("X_listの型は")
# # print(type(X_list))


# print("X2の型は")
# print(type(X))
# X=X.tolist()
# print(X)

# #PandasのDataFrameをNumPyのndarrayに変換
# print("virgicolorとvirginicaのndarray")
# oldy = oldy
# # oldy = y[(y['y'] != 0)]
# print(oldy)
# print(oldy.shape)
# print("oldyの型は")
# print(type(oldy))
# y = oldy.values
# print("yの型は")
# print(type(y))
# #訓練データ75%、検証データ25%として分割
# X_train, X_test, y_train, y_test = train_test_split(
# X, y, train_size= 0.75, test_size=0.25)
# print("sepal_lengthとpetal_lengthの訓練データ")
# print(X_train)
# print("sepal_lengthとpetal_lengthの検証データ")
# print(X_test)
# print("virgicolorとvirginicaの訓練データ")
# print(y_train)
# print("virgicolorとvirginicaの検証データ")
# print(y_test)


# #標準化は「平均が0、標準偏差が1になるようにデータを加工（スケーリング）」し、正規化は「最低が0、最高が1になるようにデータを加工（スケーリング）する」モジュールと説明されています。
# #標準化（standardization） と 正規化（normalization）
# from sklearn.preprocessing import StandardScaler
# # data = [[0, 0], [0, 0], [1, 1], [1, 1]]

# #X_train, X_test, y_train, y_test

# X_train_data = X_train #"sepal_lengthとpetal_lengthの訓練データ"
# scaler = StandardScaler()
# print(scaler.fit(X_train_data))
# # print(scaler.mean_)
# print(scaler.transform(X_train_data))

# X_test_data = X_test #"sepal_lengthとpetal_lengthの検証データ"
# scaler = StandardScaler()
# print(scaler.fit(X_test_data))
# # print(scaler.fit(X_test_data))
# # print(scaler.mean_)
# print(scaler.transform(X_test_data))


# y_train_data = y_train #"virgicolorとvirginicaの訓練データ"
# scaler = StandardScaler()
# print(scaler.fit(y_train_data))
# # print(scaler.mean_)
# print(scaler.transform(y_train_data))

# y_test_data = y_test #"virgicolorとvirginicaの検証データ"
# scaler = StandardScaler()
# print(scaler.fit(y_test_data))
# # print(scaler.fit(y_test_data))
# # print(scaler.mean_)
# print(scaler.transform(y_test_data))



# def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica']):
#     """
#     2値分類を2次元の特徴量で学習したモデルの決定領域を描く。
#     背景の色が学習したモデルによる推定値から描画される。
#     散布図の点は訓練データまたは検証データである。
#     Parameters
#     ----------------
#     X : ndarray, shape(n_samples, 2)
#         特徴量
#     y : ndarray, shape(n_samples,)
#         ラベル
#     model : object
#         学習したモデルのインスンタスを入れる
#     step : float, (default : 0.1)
#         推定値を計算する間隔を設定する
#     title : str
#         グラフのタイトルの文章を与える
#     xlabel, ylabel : str
#         軸ラベルの文章を与える
#     target_names= : list of str
#         凡例の一覧を与える
#     """
#     # setting
#     scatter_color = ['red', 'blue']
#     contourf_color = ['pink', 'skyblue']
#     n_class = 2
#     # pred
#     mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
#     mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
#     y_pred = model.predict(mesh).reshape(mesh_f0.shape)
#     # plot
#     plt.title(title)
#     plt.xlabel(xlabel)
#     plt.ylabel(ylabel)
#     plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
#     plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
#     for i, target in enumerate(set(y)):
#         plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
#     patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
#     plt.legend(handles=patches)
#     plt.legend()
#     plt.show()

# #訓練データをプロット
# # X_train_data_list = list(itertools.chain.from_iterable(X_train_data))
# # print(X_train_data_list)
# X1=scaler.transform(X_train_data)
# # print(X1)
# y1=list(itertools.chain.from_iterable(y_train_data))
# # print(y1)
# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=3)
# model = neigh.fit(X1,y1)
# decision_region(X1, y1, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])

# #検証データをプロット
# X2=scaler.transform(X_test_data)
# # print(X2)
# y2=list(itertools.chain.from_iterable(y_test_data))
# # print(y2)
# decision_region(X2, y2, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import numpy as np
import itertools

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

iris_dataset = load_iris()



#説明変数
iris_dataset_data = pd.DataFrame(iris_dataset.data, columns=["sepal_length","sepal_width","petal_length","petal_width"])
iris_dataset_all = pd.concat([iris_dataset_data], axis=1)
X = iris_dataset_all
# print(X)
print(type(X))
#目的変数（花の種類）も同様にyに格納してください。
iris_dataset_target = pd.DataFrame(iris_dataset.target, columns=["Species"])
iris_dataset_all2 = pd.concat([iris_dataset_target], axis=1)
y = iris_dataset_all2
print(type(y))
# print(y)

# df = pd.concat([X, y], axis=1)
# print(df)

#sepal_lengthとpetal_length
print("sepal_lengthとpetal_lengthのデータフレーム")
x=X.loc[:,['sepal_length','petal_length']]
print(type(x))
print(x)

oldy=y[(y['Species'] == 1) | (y['Species'] == 2)]
print(type(oldy))
print(oldy)

#PandasのDataFrameをNumPyのndarrayに変換
print("sepal_lengthとpetal_lengthのndarray")
x=x.loc[50:149,['sepal_length','petal_length']]
print(type(x))
print(x)
print("xの型は")
print(type(x))
X = x.values
print(X)
print("Xの型は")
print(type(X))
# X_list=X.tolist()
# print(X_list)
# print("X_listの型は")
# print(type(X_list))


print("X2の型は")
print(type(X))
X=X.tolist()
print(X)

#PandasのDataFrameをNumPyのndarrayに変換
print("virgicolorとvirginicaのndarray")
oldy = oldy
# oldy = y[(y['y'] != 0)]
print(oldy)
print(oldy.shape)
print("oldyの型は")
print(type(oldy))
y = oldy.values
print("yの型は")
print(type(y))
#訓練データ75%、検証データ25%として分割
X_train, X_test, y_train, y_test = train_test_split(
X, y, train_size= 0.75, test_size=0.25)
print("sepal_lengthとpetal_lengthの訓練データ")
print(X_train)
print("sepal_lengthとpetal_lengthの検証データ")
print(X_test)
print("virgicolorとvirginicaの訓練データ")
print(y_train)
print("virgicolorとvirginicaの検証データ")
print(y_test)


#標準化は「平均が0、標準偏差が1になるようにデータを加工（スケーリング）」し、正規化は「最低が0、最高が1になるようにデータを加工（スケーリング）する」モジュールと説明されています。
#標準化（standardization） と 正規化（normalization）
from sklearn.preprocessing import StandardScaler
# data = [[0, 0], [0, 0], [1, 1], [1, 1]]

#X_train, X_test, y_train, y_test

X_train_data = X_train #"sepal_lengthとpetal_lengthの訓練データ"
scaler = StandardScaler()
print(scaler.fit(X_train_data))
# print(scaler.mean_)
print(scaler.transform(X_train_data))

X_test_data = X_test #"sepal_lengthとpetal_lengthの検証データ"
scaler = StandardScaler()
print(scaler.fit(X_test_data))
# print(scaler.fit(X_test_data))
# print(scaler.mean_)
print(scaler.transform(X_test_data))


y_train_data = y_train #"virgicolorとvirginicaの訓練データ"
scaler = StandardScaler()
print(scaler.fit(y_train_data))
# print(scaler.mean_)
print(scaler.transform(y_train_data))

y_test_data = y_test #"virgicolorとvirginicaの検証データ"
scaler = StandardScaler()
print(scaler.fit(y_test_data))
# print(scaler.fit(y_test_data))
# print(scaler.mean_)
print(scaler.transform(y_test_data))



def decision_region(X, y, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica']):
    """
    2値分類を2次元の特徴量で学習したモデルの決定領域を描く。
    背景の色が学習したモデルによる推定値から描画される。
    散布図の点は訓練データまたは検証データである。
    Parameters
    ----------------
    X : ndarray, shape(n_samples, 2)
        特徴量
    y : ndarray, shape(n_samples,)
        ラベル
    model : object
        学習したモデルのインスンタスを入れる
    step : float, (default : 0.1)
        推定値を計算する間隔を設定する
    title : str
        グラフのタイトルの文章を与える
    xlabel, ylabel : str
        軸ラベルの文章を与える
    target_names= : list of str
        凡例の一覧を与える
    """
    # setting
    scatter_color = ['red', 'blue']
    contourf_color = ['pink', 'skyblue']
    n_class = 2
    # pred
    mesh_f0, mesh_f1  = np.meshgrid(np.arange(np.min(X[:,0])-0.5, np.max(X[:,0])+0.5, step), np.arange(np.min(X[:,1])-0.5, np.max(X[:,1])+0.5, step))
    mesh = np.c_[np.ravel(mesh_f0),np.ravel(mesh_f1)]
    y_pred = model.predict(mesh).reshape(mesh_f0.shape)
    # plot
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.contourf(mesh_f0, mesh_f1, y_pred, n_class-1, cmap=ListedColormap(contourf_color))
    plt.contour(mesh_f0, mesh_f1, y_pred, n_class-1, colors='y', linewidths=3, alpha=0.5)
    for i, target in enumerate(set(y)):
        plt.scatter(X[y==target][:, 0], X[y==target][:, 1], s=80, color=scatter_color[i], label=target_names[i], marker='o')
    patches = [mpatches.Patch(color=scatter_color[i], label=target_names[i]) for i in range(n_class)]
    plt.legend(handles=patches)
    plt.legend()
    plt.show()

#訓練データをプロット
# X_train_data_list = list(itertools.chain.from_iterable(X_train_data))
# print(X_train_data_list)
X1=scaler.transform(X_train_data)
# print(X1)
y1=list(itertools.chain.from_iterable(y_train_data))
# print(y1)
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
model = neigh.fit(X1,y1)
decision_region(X1, y1, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])

#検証データをプロット
X2=scaler.transform(X_test_data)
# print(X2)
y2=list(itertools.chain.from_iterable(y_test_data))
# print(y2)
decision_region(X2, y2, model, step=0.01, title='decision region', xlabel='xlabel', ylabel='ylabel', target_names=['versicolor', 'virginica'])


In [ ]:
#スクラッチSVM実装でシンプルデータセット１を使用して、学習と正答率を算出する

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pprint
from sklearn import linear_model, metrics, preprocessing #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用

#シンプルデータセット1作成コード
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, n_samples // 2)
f1 = np.random.multivariate_normal(f1, cov, n_samples // 2)
X1_1 = np.concatenate([f0, f1])
y1_1= np.concatenate([
    np.full(n_samples // 2, 1),
    np.full(n_samples // 2, -1)
])

print("X1_1")
print(X1_1.shape)
print("y1_1")
print(y1_1.shape)


#トレーニングデータとテストデータに分けて実行してみる------------------
X_train, X_test, y_train, y_test=train_test_split(X1_1, y1_1, train_size=0.8)



scaler = StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

import numpy as np
# X = [10 ,10]
X=X_train
print("X1")
print(X_train)
# print(X.shape)
X = np.array(X)
print("X2")
print(X.T)
print(X.T.shape)
h = X @ X.T
print("h")
print(h)
print(h.shape)